# Chapter 10 SEQUENCE MODELING: RECURRENT AND RECURSIVE NETS

* 유주원

# Chapter 10.8 Echo State Networks

* 이전 hidden layer에서 다음 hidden layer로 갈 때의 weights와 input layer에서 hidden layer로 갈 때의 weight는 RNN에서 학습하기에 어려운 파라미터 중 하나이다.

### Echo State Network
* 2003 Jaeger에 의해 발표된 모델.
* 오직 output 뉴런과 hidden 뉴런 사이의 weight만 학습한다.
* 최초의 weight를 initialize할 때 weight vector들이 서로 orthogonal 하도록 설정한다.
* 기존 RNN에 비해 학습 속도가 빠르고, 성능도 의외로 잘나온다.
* https://docs.google.com/viewer?url=https%3A%2F%2Fd396qusza40orc.cloudfront.net%2Fneuralnets%2Flecture_slides%2Flec8.pptx
<img src="./figures/fig1.png" />

### liquid state machines
* ESN 처럼 연속적인 값의 hidden unit을 사용하는 것이 아니라, spiking neuron을 사용한다.
* 위의 차이점만 빼고 나머지는 ESN과 상당히 유사함.

* ESN과 liquid state machines를 reservoir computing이라고 정의했는데, 이는 hidden unit이 input의 history에 대해 다른 측면을 찾아낼 수 있다는 것을 나타낸다.

### reservoir computing
* 이러한 reservoir computing recurrent network는 kernal machine과 상당히 유사하다.
* 임의의 길이(시간 t까지의 input history)를 고정된 길이의 벡터(recurrent state h)로 매핑을 해주고, 선형 예측기(선형 회귀)에서는 문제를 풀기 위해 매핑이 적용될 수 있다.
* training 기준은 output weight의 함수로써 convex하게 디자인 될 수 있다.
* 예를 들어 만약 output이 hidden unit에서 output target으로 향하는 선형 회귀로 구성이 되어 있고, training 기준이 MSE라면 이 모델은 convex하고, 단순한 학습 알고리즘에 따라 문제를 해결할 수가 있다.

### how do we set the input and recurrent weights?
* recurrent net을 dynamic system으로 보자
* 입력과 recurrent에 대한 weight를 dynamic system의 edge of stability에 가깝도록 설정하자.
* 기본 개념은 Jacobian의 eigenvalue값을 1에 가깝도록 만드는 것이다.
* recurrent network의 중요한 특징이 바로 jacobian  $J^{(t)} = ∂s^{(t)}/ ∂s^{(t-1)}$의 eigenvalue spectrum(고유값 집합)이다.
* 특히 중요한 것은 $J^{(t)}$의 spectral radius이며, eigenvalue의 절대 값의 최대값으로 정의된다.
### spectral radius
* spectral radius의 효과를 이해하기 위해, 변하지 않는 t를 가진 Jacobian matrix J와 함께 backpropagation을 살펴보자.
* 해당 예제는 네트워크가 순수 선형일 경우에만 적용된다.
* J가 eigenvalue λ와 상응하는 eigenvector v를 가지고 있다고 가정하자.
* 만약 우리가 gradient vector g를 가지고 시작을 했고, back-propagation의 1스텝이 지난 후라면, 우리는 Jg를 가지게 되고 n step 후에는 $J^{n}g$가 된다.
* 다시 처음으로 와서 이제 우리가 gradient vector g에 eigenvector를 더한 값을 가지고 시작했고, back-propagation의 1스텝이 지난 후라면, 우리는 J(g+δv)를 가지게 되고 n step 후에는 $J^{n}(g+δv)$ 값을 가지게 될 것이다.
* 이를 통해 우리는 n step의 back-propagation 후에 $δJ^{n}v$ 만큼의 값이 추가된다는 것을 확인할 수가 있다.
* 만약 v가 eigenvalue λ 가진  J의 고유 단위 벡터로 선택이 되었다면, Jacobian의 곱은 단순히 각 스텝에서의 차이를 스케일링 한다.
* 위에 언급한 두가지 back-propagation은 $δ|λ|^{n}$의 거리만큼 떨어져있다.
* |λ| > 1일때, 편차의 크기 $δ|λ|^{n}$는 기하급수적으로 커지고, |λ| < 1 일 때는 기하급수적으로 작아진다.
* 위의 경우는 Jacobian이 비선형이 아닌 recurrent network 상에서 매 time step에 동일하다는 가정이 전제해야 한다.
* 비선형성이 존재할때, 비선형의 미분이 많은 time step에서 점점 0에 가까워질 것이다. spectral radius를 통해 이를 예방할 수가 있다. 
* http://aikorea.org/blog/rnn-tutorial-3/

### Jacobian matrix
* Jacobian matrix는 어떻게 $h^{(t)}$가 on step forward 하는지, 또는 어떻게 $h^{(t+1)}$ 에 대한 gradient가 back-propagation 되는지를 말해준다.
* W와 J는 symmetric 할 필요가 없기 때문에, 복소수의 eigenvalue와 eigenvector 를 가질 수 있다.
* 심지어 back-propagation 내에서 $h^{(t)}$ 또는 $h^{(t)}$의 작은 변화가 실수였어도, 복소수 처럼 표현 될 수 있다.
* 중요한 것은 벡터를 사용하여 행렬을 곱했을 때, 이러한 복소수 기본 계수의 크기에 어떤 일이 발생하는가이다.
* eigenvalue의 크기가 1보다 크면 확대 또는 축소될 것이다.

* 비선형 맵에서 Jacobian은 각 step에서의 변경이 용이하고 그래서 더 복잡하다.
* 하지만 작은 초기의 변화가 몇 단계 후에는 큰 변화로 변경될 수 있다는 것은 사실이다.


* echo state network의 전략은 단순하게 spectral radius를 3으로 가지게 하기 위해 weight를 수정하는 것이다.
* 왜 3?  tanh같은 비선형의 안정적인 영향 덕분에 exploding 되지 않으면서 정보를 전달할 수가 있다.
* 최근에는 ESN에서 weight를 설정하는데 사용했던 기술을 fully trainable recurrent network의 weight 초기화에 사용함으로써 long-term dependency를 학습하는데 도움이 되었다.
* 초기 spectral radius를 1.2로 설정한 후, section 8.4 (parameter initialization strategies)와 함께 결합하면 잘 동작한다.